In [49]:
from myutils import Datasets
import tensorflow as tf
from matplotlib import pyplot as plt
from myutils import Helpers as h
import numpy as np
import os
from sklearn.decomposition import PCA
from myutils import Preprocessing as pp
from myutils import Complexity_Measures as cm
import itertools
from random import randint


#Magic Command, so changes in myutils module are reloaded
%load_ext autoreload
%autoreload 1
%aimport myutils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import Data

In [50]:
data = Datasets.get_preprocessed_datasets("RAW","PCA")
raw = data["RAW"]
pca = data["PCA"]

In [20]:
# choose random 6 subsets
np.random.seed(42)

randomset = list()
for c in np.random.choice(45, size=6):
    randomset.append(os.listdir("data/PCA")[c])

randomset.append("6,9")
randomset

['1,9', '6,8', '1,4', '1,6', '4,7', '3,6', '6,9']

# First find sweet spot of AE Epochs

In [28]:
for subsetname in randomset:
    for epochs in range(1,30,2):
        print(subsetname, epochs)
        x_train = raw[subsetname]["x_train"]
        x_test = raw[subsetname]["x_test"]

        #autoencoder
        x_train_ae,x_test_ae,hist= pp.Autoencoder(x_train=x_train, x_test=x_test, outputsize=4,epochs=epochs)
        h.store(x_train_ae,"data/ae_epochs/epochs={}/{}".format(epochs,subsetname),"x_train_ae")
        h.store(x_test_ae,"data/ae_epochs/epochs={}/{}".format(epochs,subsetname),"x_test_ae")
        h.store(hist,"data/ae_epochs/epochs={}/{}".format(epochs,subsetname),"hist")

1,9 1
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

397/397 [==============================] - 3s 8ms/step - loss: 0.0500
1,9 3
Epoch 1/3
397/397 [==============================] - 4s 8ms/step - loss: 0.0500
Epoch 2/3
397/397 [==============================] - 3s 8ms/step - loss: 0.0422
Epoch 3/3
397/397 [==============================] - 3s 8ms/step - loss: 0.0355
1,9 5
Epoch 1/5
397/397 [==============================] - 3s 8ms/step - loss: 0.0500
Epoch 2/5
397/397 [==============================] - 3s 7ms/step - loss: 0.0422
Epoch 3/5
397/397 [==============================] - 3s 8ms/step - loss: 0.0355
Epoch 4/5
397/397 [==============================] - 3s 7ms/step - loss: 0.0342
Epoch 5/5
397/397 [==============================] - 3s 8ms/step - loss: 0.0320
1,9 7
Epoch 1/7
397/397 [==============================] - 4s 8ms/step - loss: 0.0500
Epoch 2/7
397/397 [==============================] - 3s 8ms/step - loss: 0.0422
Epoch 3/7
397/397 [======

In [32]:
raw = Datasets.get_preprocessed_datasets("RAW")["RAW"]

In [35]:
for epochs_folder in sorted(os.listdir("data/ae_epochs"),key=lambda x: int(x.split("=")[-1])):


    for subsetname in os.listdir("data/ae_epochs/{}".format(epochs_folder)):

        print(epochs_folder, subsetname)
        x_train = np.load("data/ae_epochs/{}/{}/x_train_ae.npy".format(epochs_folder,subsetname))
        x_test = np.load("data/ae_epochs/{}/{}/x_test_ae.npy".format(epochs_folder,subsetname))

        tf.compat.v1.reset_default_graph()
        with tf.device("/gpu:0"):
              
            model  = tf.keras.models.Sequential([
                tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(4,))])


            model.compile(optimizer='adam',loss="binary_crossentropy", metrics=['accuracy'])

            hist = model.fit(
                x=x_train,
                y=raw[subsetname]["y_train_binary"],
                epochs=10,
                batch_size = 32,
                verbose=0,
                validation_data=(x_test,raw[subsetname]["y_test_binary"])).history

        weights = model.trainable_weights[0].numpy()
        bias = model.trainable_weights[1].numpy()

        h.store(weights,"measure/ae_epochs/{}/{}".format(epochs_folder,subsetname),"weights")
        h.store(bias,"measure/ae_epochs/{}/{}".format(epochs_folder,subsetname),"bias")
        h.store(hist,"measure/ae_epochs/{}/{}".format(epochs_folder,subsetname),"hist")

        del model
        

       

epochs=1 4,7
epochs=1 1,4
epochs=1 3,6
epochs=1 6,8
epochs=1 6,9
epochs=1 1,9
epochs=1 1,6
epochs=3 4,7
epochs=3 1,4
epochs=3 3,6
epochs=3 6,8
epochs=3 6,9
epochs=3 1,9
epochs=3 1,6
epochs=5 4,7
epochs=5 1,4
epochs=5 3,6
epochs=5 6,8
epochs=5 6,9
epochs=5 1,9
epochs=5 1,6
epochs=7 4,7
epochs=7 1,4
epochs=7 3,6
epochs=7 6,8
epochs=7 6,9
epochs=7 1,9
epochs=7 1,6
epochs=9 4,7
epochs=9 1,4
epochs=9 3,6
epochs=9 6,8
epochs=9 6,9
epochs=9 1,9
epochs=9 1,6
epochs=11 4,7
epochs=11 1,4
epochs=11 3,6
epochs=11 6,8
epochs=11 6,9
epochs=11 1,9
epochs=11 1,6
epochs=13 4,7
epochs=13 1,4
epochs=13 3,6
epochs=13 6,8
epochs=13 6,9
epochs=13 1,9
epochs=13 1,6
epochs=15 4,7
epochs=15 1,4
epochs=15 3,6
epochs=15 6,8
epochs=15 6,9
epochs=15 1,9
epochs=15 1,6
epochs=17 4,7
epochs=17 1,4
epochs=17 3,6
epochs=17 6,8
epochs=17 6,9
epochs=17 1,9
epochs=17 1,6
epochs=19 4,7
epochs=19 1,4
epochs=19 3,6
epochs=19 6,8
epochs=19 6,9
epochs=19 1,9
epochs=19 1,6
epochs=21 4,7
epochs=21 1,4
epochs=21 3,6
epochs=21 6,8

# Store different numbers of Autoencoder Latentdim and PCA Components

In [44]:
for subsetname in randomset:
    for latentdim in range(1,12,1):

        print(subsetname,latentdim)
        x_train = raw[subsetname]["x_train"]
        x_test = raw[subsetname]["x_test"]

        #apply preprocessings
        #pca
        x_train_pca,x_test_pca = pp.PCA(x_train=x_train, x_test=x_test, outputsize=latentdim)

        h.store(x_train_pca,"data/latentdimvergleich/pca/latentdim={}/{}".format(latentdim,subsetname),"x_train_pca")
        h.store(x_test_pca,"data/latentdimvergleich/pca/latentdim={}/{}".format(latentdim,subsetname),"x_test_pca")

        #autoencoder
        x_train_ae,x_test_ae,hist= pp.Autoencoder(x_train=x_train, x_test=x_test, outputsize=latentdim,epochs=6)
        h.store(x_train_ae,"data/latentdimvergleich/ae/latentdim={}/{}".format(latentdim,subsetname),"x_train_ae")
        h.store(x_test_ae,"data/latentdimvergleich/ae/latentdim={}/{}".format(latentdim,subsetname),"x_test_ae")
        h.store(hist,"data/latentdimvergleich/ae/latentdim={}/{}".format(latentdim,subsetname),"hist")
    

4,7 6
4,7 8
4,7 9


In [55]:
for subsetname in randomset:
    for latentdim in range(1,12,1):

        print(subsetname,latentdim)
        x_train = raw[subsetname]["x_train"]
        x_test = raw[subsetname]["x_test"]
        #autoencoder
        x_train_ae,x_test_ae,hist= pp.Autoencoder(x_train=x_train, x_test=x_test, outputsize=latentdim,epochs=7)
        h.store(x_train_ae,"data/latentdimvergleich/ae/epochs={}/latentdim={}/{}".format(7,latentdim,subsetname),"x_train_ae")
        h.store(x_test_ae,"data/latentdimvergleich/ae/epochs={}/latentdim={}/{}".format(7,latentdim,subsetname),"x_test_ae")
        h.store(hist,"data/latentdimvergleich/ae/epochs={}/latentdim={}/{}".format(7,latentdim,subsetname),"hist")
    

1,9 1
1,9 2
1,9 3
1,9 4
1,9 5
1,9 6
1,9 7
1,9 8
1,9 9
1,9 10
1,9 11
6,8 1
6,8 2
6,8 3
6,8 4
6,8 5
6,8 6
6,8 7
6,8 8
6,8 9
6,8 10
6,8 11
1,4 1
1,4 2
1,4 3
1,4 4
1,4 5
1,4 6
1,4 7
1,4 8
1,4 9
1,4 10
1,4 11
1,6 1
1,6 2
1,6 3
1,6 4
1,6 5
1,6 6
1,6 7
1,6 8
1,6 9
1,6 10
1,6 11
4,7 1
4,7 2
4,7 3
4,7 4
4,7 5
4,7 6
4,7 7
4,7 8
4,7 9
4,7 10
4,7 11
3,6 1
3,6 2
3,6 3
3,6 4
3,6 5
3,6 6
3,6 7
3,6 8
3,6 9
3,6 10
3,6 11
6,9 1
6,9 2
6,9 3
6,9 4
6,9 5
6,9 6
6,9 7
6,9 8
6,9 9
6,9 10
6,9 11


# Now run neural Network

In [79]:
for preprocessing in os.listdir("data/latentdimvergleich(7)"):
    for latentdim in os.listdir("data/latentdimvergleich(7)/{}".format(preprocessing)):
        latentdimnum = int(latentdim.split("=")[-1])
        for subset in os.listdir("data/latentdimvergleich(7)/{}/{}".format(preprocessing,latentdim)):

            print(preprocessing,latentdim,subset)
        
            x_train = np.load("data/latentdimvergleich(7)/{}/{}/{}/x_train_{}.npy".format(preprocessing,latentdim,subset,preprocessing))
            x_test = np.load("data/latentdimvergleich(7)/{}/{}/{}/x_test_{}.npy".format(preprocessing,latentdim,subset,preprocessing))

            tf.compat.v1.reset_default_graph()
            with tf.device("/gpu:0"):

               if latentdimnum > 1:
                    model = tf.keras.models.Sequential([
                         tf.keras.layers.Dense(round(latentdimnum/2), activation='relu', input_shape=(latentdimnum,)),
                         tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(latentdimnum,))])

               else:
                    model  = tf.keras.models.Sequential([
                         tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(latentdimnum,))])
    
               model.compile(optimizer='adam',loss="binary_crossentropy", metrics=['accuracy'])

               hist = model.fit(
                    x=x_train,
                    y=raw[subset]["y_train_binary"],
                    epochs=10,
                    batch_size = 32,
                    verbose=0,
                    validation_data=(x_test,raw[subset]["y_test_binary"])).history
            
            h.store(hist,"measure/latentdimvergleich(7)_bigger/{}/{}/{}".format(preprocessing,latentdim,subset),"hist")

            weights = model.trainable_weights[0].numpy()
            bias = model.trainable_weights[1].numpy()

            h.store(weights,"measure/latentdimvergleich(7)_bigger/{}/{}/{}".format(preprocessing,latentdim,subset),"weights")
            h.store(bias,"measure/latentdimvergleich(7)_bigger/{}/{}/{}".format(preprocessing,latentdim,subset),"bias")

            del model

pca latentdim=2 4,7
pca latentdim=2 1,4
pca latentdim=2 3,6
pca latentdim=2 6,8
pca latentdim=2 6,9
pca latentdim=2 1,9
pca latentdim=2 1,6
pca latentdim=5 4,7
pca latentdim=5 1,4
pca latentdim=5 3,6
pca latentdim=5 6,8
pca latentdim=5 6,9
pca latentdim=5 1,9
pca latentdim=5 1,6
pca latentdim=4 4,7
pca latentdim=4 1,4
pca latentdim=4 3,6
pca latentdim=4 6,8
pca latentdim=4 6,9
pca latentdim=4 1,9
pca latentdim=4 1,6
pca latentdim=3 4,7
pca latentdim=3 1,4
pca latentdim=3 3,6
pca latentdim=3 6,8
pca latentdim=3 6,9
pca latentdim=3 1,9
pca latentdim=3 1,6
pca latentdim=10 4,7
pca latentdim=10 1,4
pca latentdim=10 3,6
pca latentdim=10 6,8
pca latentdim=10 6,9
pca latentdim=10 1,9
pca latentdim=10 1,6
pca latentdim=11 4,7
pca latentdim=11 1,4
pca latentdim=11 3,6
pca latentdim=11 6,8
pca latentdim=11 6,9
pca latentdim=11 1,9
pca latentdim=11 1,6
pca latentdim=6 4,7
pca latentdim=6 1,4
pca latentdim=6 3,6
pca latentdim=6 6,8
pca latentdim=6 6,9
pca latentdim=6 1,9
pca latentdim=6 1,6
pca la

In [73]:

for latentdim in os.listdir("data/latentdimvergleich/ae/epochs=7/"):
    latentdimnum = int(latentdim.split("=")[-1])
    for subset in os.listdir("data/latentdimvergleich/ae/epochs=7/{}".format(latentdim)):

        print(latentdim,subset)
    
        x_train = np.load("data/latentdimvergleich/ae/epochs=7/{}/{}/x_train_ae.npy".format(latentdim,subset))
        x_test = np.load("data/latentdimvergleich/ae/epochs=7/{}/{}/x_test_ae.npy".format(latentdim,subset))

        tf.compat.v1.reset_default_graph()
        with tf.device("/gpu:0"):

                if latentdimnum > 1:
                    model  = tf.keras.models.Sequential([
                        tf.keras.layers.Dense(round(latentdimnum/2), activation='relu', input_shape=(latentdimnum,)),
                        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(latentdimnum,))])
                
                model  = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(latentdimnum,))])

                model.compile(optimizer='adam',loss="binary_crossentropy", metrics=['accuracy'])

                hist = model.fit(
                    x=x_train,
                    y=raw[subset]["y_train_binary"],
                    epochs=10,
                    batch_size = 32,
                    verbose=0,
                    validation_data=(x_test,raw[subset]["y_test_binary"])).history
        
        h.store(hist,"measure/latentdimvergleich/ae/epochs=7/{}/{}".format(latentdim,subset),"hist")

        weights = model.trainable_weights[0].numpy()
        bias = model.trainable_weights[1].numpy()

        h.store(weights,"measure/latentdimvergleich/ae/epochs=7/{}/{}".format(latentdim,subset),"weights")
        h.store(bias,"measure/latentdimvergleich/ae/epochs=7/{}/{}".format(latentdim,subset),"bias")

        del model

latentdim=2 4,7
latentdim=2 1,4
latentdim=2 3,6
latentdim=2 6,8
latentdim=2 6,9
latentdim=2 1,9
latentdim=2 1,6
latentdim=5 4,7
latentdim=5 1,4
latentdim=5 3,6
latentdim=5 6,8
latentdim=5 6,9
latentdim=5 1,9
latentdim=5 1,6
latentdim=4 4,7
latentdim=4 1,4
latentdim=4 3,6
latentdim=4 6,8
latentdim=4 6,9
latentdim=4 1,9
latentdim=4 1,6
latentdim=3 4,7
latentdim=3 1,4
latentdim=3 3,6
latentdim=3 6,8
latentdim=3 6,9
latentdim=3 1,9
latentdim=3 1,6
latentdim=10 4,7
latentdim=10 1,4
latentdim=10 3,6
latentdim=10 6,8
latentdim=10 6,9
latentdim=10 1,9
latentdim=10 1,6
latentdim=11 4,7
latentdim=11 1,4
latentdim=11 3,6
latentdim=11 6,8
latentdim=11 6,9
latentdim=11 1,9
latentdim=11 1,6
latentdim=6 4,7
latentdim=6 1,4
latentdim=6 3,6
latentdim=6 6,8
latentdim=6 6,9
latentdim=6 1,9
latentdim=6 1,6
latentdim=1 4,7
latentdim=1 1,4
latentdim=1 3,6
latentdim=1 6,8
latentdim=1 6,9
latentdim=1 1,9
latentdim=1 1,6
latentdim=8 4,7
latentdim=8 1,4
latentdim=8 3,6
latentdim=8 6,8
latentdim=8 6,9
latentdim=

In [76]:
round(3/2)

2